In [2]:
import random
import numpy as np
import json
import os
from shapely.geometry import Polygon
from ezdxf import new
from ezdxf.math import Vec3
import matplotlib.pyplot as plt

In [3]:
np.set_printoptions(precision=6, suppress=True)
NP_FLOAT_DTYPE = np.float32

class Config:
    def __init__(self):
        self.num_polygons = 30
        self.coord_range = (0, 80)
        self.image_size = 3000
        self.max_points = 8
        self.min_area = 3000
        self.skip_center_percent = 30
        self.output_dirs = {
            "dxf": "./generated_db/dxf",
            "json": "./generated_db/json",
            "png": "./generated_db/png",
            "numpy": "./generated_db/numpy"
        }

class PlotGenerator:
    def __init__(self, config):
        self.config = config
        
    def generate_valid_polygon(self, skip_center_percent=30):
        min_x, max_x = self.config.coord_range
        range_x = max_x - min_x
        skip_width = range_x * (skip_center_percent / 100)
        left_zone_max = min_x + (range_x - skip_width) / 2
        right_zone_min = max_x - (range_x - skip_width) / 2
        points = []
        for _ in range(random.randint(4, self.config.max_points)):
            x = random.randint(min_x, int(left_zone_max)) if random.random() < 0.5 else random.randint(int(right_zone_min), max_x)
            y = random.randint(min_x, int(left_zone_max)) if random.random() < 0.5 else random.randint(int(right_zone_min), max_x)
            points.append([x, y])
        center = np.mean(points, axis=0)
        angles = np.arctan2(np.array(points)[:,1] - center[1], np.array(points)[:,0] - center[0])
        points = np.array(points)[np.argsort(angles)]
        polygon = Polygon(points)
        if not polygon.is_valid or polygon.area < self.config.min_area:
            return self.generate_valid_polygon(skip_center_percent)
        return points.astype(NP_FLOAT_DTYPE)

class FileSaver:
    def __init__(self, config):
        self.config = config
    
    def save_as_dxf(self, points, filename):
        doc = new(setup=True)
        msp = doc.modelspace()
        valid_points = points[~np.isnan(points).any(axis=1)]
        dxf_points = [Vec3(float(x), float(y)) for x, y in valid_points]
        dxf_points.append(dxf_points[0])
        msp.add_lwpolyline(dxf_points)
        doc.saveas(filename)
    
    def save_as_json(self, points, filename):
        valid_points = points[~np.isnan(points).any(axis=1)]
        data = {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [valid_points.tolist() + valid_points[0].tolist()]
            },
            "properties": {
                "num_points": len(valid_points)
            }
        }
        with open(filename, 'w') as f:
            json.dump(data, f, indent=2)
    
    def save_as_png(self, points, filename):
        valid_points = points[~np.isnan(points).any(axis=1)]
        fig, ax = plt.subplots(figsize=(30, 30), dpi=100)
        ax.set_xlim(0, self.config.image_size)
        ax.set_ylim(0, self.config.image_size)
        ax.axis('off')
        ax.set_facecolor('black')
        fig.patch.set_facecolor('black')
        scale_factor = self.config.image_size / self.config.coord_range[1]
        points_scaled = valid_points * scale_factor
        polygon = plt.Polygon(points_scaled, fill=False, edgecolor='white', linewidth=3)
        ax.add_patch(polygon)
        plt.savefig(filename, bbox_inches='tight', pad_inches=0)
        plt.close()
    
    def save_as_numpy(self, points, filename):
        valid_points = points[~np.isnan(points).any(axis=1)]
        data = {
            "plot_points": valid_points.astype(NP_FLOAT_DTYPE),
            "metadata": {
                "num_points": len(valid_points)
            }
        }
        np.save(filename, data, allow_pickle=True)

class GeneratorApp:
    def __init__(self):
        self.config = Config()
        self.plot_gen = PlotGenerator(self.config)
        self.file_saver = FileSaver(self.config)
        
    def generate_all_files(self):
        for dir_path in self.config.output_dirs.values():
            os.makedirs(dir_path, exist_ok=True)
        successful = 0
        file_index = 1
        while successful < self.config.num_polygons:
            poly = self.plot_gen.generate_valid_polygon(self.config.skip_center_percent)
            self._save_files(poly, file_index)
            successful += 1
            file_index += 1
        print(f"Успешно сохранено {successful} участков")
    
    def _save_files(self, poly, file_index):
        base_path = f"land_plot_{file_index}"
        dxf_path = os.path.join(self.config.output_dirs["dxf"], f"{base_path}.dxf")
        self.file_saver.save_as_dxf(poly, dxf_path)
        json_path = os.path.join(self.config.output_dirs["json"], f"{base_path}.json")
        self.file_saver.save_as_json(poly, json_path)
        png_path = os.path.join(self.config.output_dirs["png"], f"{base_path}.png")
        self.file_saver.save_as_png(poly, png_path)
        numpy_path = os.path.join(self.config.output_dirs["numpy"], f"{base_path}.npy")
        self.file_saver.save_as_numpy(poly, numpy_path)

if __name__ == "__main__":
    app = GeneratorApp()
    app.generate_all_files()


Успешно сохранено 30 участков
